In [6]:
from imp import reload
import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import random
import datetime
from  tqdm import tqdm

import data_generator
import hparams
import model_unet
import numpy as np
from loss_function import CrossEntropyLoss_Origin
import utils
reload(utils)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [ ]:
def train(dataloader, model, loss_fn, optimizer, scheduler, out_floor):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader): # 每次返回一个batch
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X, out_floor)
        
        if out_floor == 0:
            loss = loss_fn(pred, y)
        else:
            # downsample y
            y_downsample = utils.downsample(y, out_floor)
            loss = loss_fn(pred, y_downsample)
            
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Avg loss: {loss/size:>7f}  [{current:>5d}/{size:>5d}]")
    scheduler.step()
            
def test(dataloader, model, out_floor):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X, out_floor)
            
            if out_floor == 0:
                loss = loss_fn(pred, y)
            else:
                # downsample y
                y_downsample = utils.downsample(y, out_floor)
                loss = loss_fn(pred, y_downsample)
            
            test_loss += loss.item()
            #correct += (pred.argmax(1) == y).type(torch.float).sum().item() # TODO
    test_loss /= size
    #correct /= size
    #print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"Test Error: Avg loss: {test_loss:>8f} \n")
    return test_loss

# split data, generate train/test_dataloader

In [2]:
'''
fold_index = list(range(10))
random.shuffle(fold_index)
test_fold_index = fold_index[0]
validation_fold_index = fold_index[1]
train_fold_index_list = fold_index[2:]
'''
train_fold_index_list = hparams.train_set_fold_index
valid_fold_index_list = hparams.validation_set_fold_index
test_fold_index_list = hparams.test_set_fold_index

# have a try :)  Train

In [ ]:
# prepare

print(f'{datetime.datetime.now()} - Preparing train_dataloader...')
train_dataloader = data_generator.source_index_to_chunk_list(source_list=train_fold_index_list)#[0:32]
train_dataloader = DataLoader(train_dataloader, batch_size=1, shuffle=True)

In [3]:
print(f'{datetime.datetime.now()} - Preparing valid_dataloader...')
valid_dataloader = data_generator.source_index_to_chunk_list(source_list=valid_fold_index_list)[0:32]
valid_dataloader = DataLoader(valid_dataloader, batch_size=2, shuffle=True)


test_dataloader = data_generator.source_index_to_chunk_list(source_list=test_fold_index_list)#[0:32]
test_dataloader = DataLoader(test_dataloader, batch_size=1, shuffle=True)

2021-05-31 21:12:28.270292 - Preparing valid_dataloader...


In [4]:
model = model_unet.UNet(device=device)
model.load_state_dict(torch.load('./train_result_floor_3/0526_f3_2/model_lr0.0001_floor3_epoch2.pth'))


model2 = model_unet.UNet(device=device)
model2.load_state_dict(torch.load('./train_result_floor_3/0526_f3/B_model_lr0.001_floor3_epoch3.pth'))
'''
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
scheduler_decay = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.94, verbose=True)
scheduler_stop = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', threshold=1e-4, factor=-1, patience=10)
'''

loss_fn = CrossEntropyLoss_Origin().to(device)

epochs = 10
num_floor = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------{datetime.datetime.now()}")
    train(train_dataloader, model, loss_fn, optimizer, scheduler_decay, num_floor)
    test_loss = test(valid_dataloader, model, num_floor)
    scheduler_stop.step(test_loss)
    if optimizer.state_dict()['param_groups'][0]['lr']<0:
        print(f'Early stop after {t} epochs.')
        break
    torch.save(model.state_dict(), f'model_saved/model_floor3_epoch{t}.pth')
print("Done!")

# 逐个比较损失

In [17]:
import evaluate
reload(evaluate)

out_floor = 3

loss_fn = CrossEntropyLoss_Origin().to(device)

model.eval()
model2.eval()

model_loss = 0
model_loss_2 = 0


with torch.no_grad():
    for a,b in valid_dataloader:
        X, y = a.to(device), b.to(device)

        Xpred = model(X, out_floor)
        y_downsample = utils.downsample(y, out_floor)
        
        clean = utils.salience_to_output(Xpred)
        
        print(evaluate.evaluate(clean, y_downsample, out_floor))
        
        '''temp = Xpred[0,0].detach().to('cpu').numpy()

        clean = utils.salience_to_output(temp, threshold=0)

        Xpred2 = model2(X, out_floor)

        temp2 = Xpred2[0,0].detach().to('cpu').numpy()

        clean2 = utils.salience_to_output(temp2, threshold=0)

        model_loss += loss_fn(Xpred, y_downsample)
        model_loss_2 += loss_fn(Xpred2, y_downsample)

        if True:#loss_fn(Xpred, y_downsample) < loss_fn(Xpred2, y_downsample):

            plt.subplot(231)
            plt.imshow(y_downsample[0].detach().to('cpu').numpy(), origin='lower')
            plt.subplot(232)
            plt.imshow(temp, origin='lower')
            plt.subplot(233)
            plt.imshow(clean, origin='lower')

            plt.subplot(234)
            plt.imshow(y_downsample[0].detach().to('cpu').numpy(), origin='lower')
            plt.subplot(235)
            plt.imshow(temp2, origin='lower')
            plt.subplot(236)
            plt.imshow(clean2, origin='lower')
            plt.show()
            print(loss_fn(Xpred, y_downsample) , loss_fn(Xpred2, y_downsample))'''

        break

torch.Size([2, 1, 45, 33]) torch.Size([2, 45, 33])
(0.015151515151515152, 1.0, 1.0, 0.038461538461538464, 0.038461538461538464)


In [ ]:
loss_fn = CrossEntropyLoss_Origin().to(device)

model.eval()
model2.eval()

X, y = None, None
for a,b in tqdm(train_dataloader):
    X, y = a.to(device), b.to(device)

    Xpred = model(X, 2)
    y_downsample = utils.downsample(y, 2)

    temp = Xpred[0,0].detach().to('cpu').numpy()

    clean = utils.salience_to_output(temp, threshold=0)


    Xpred2 = model2(X, 2)

    temp2 = Xpred2[0,0].detach().to('cpu').numpy()

    clean2 = utils.salience_to_output(temp2, threshold=0)
    
    if loss_fn(Xpred, y_downsample) + 10 < loss_fn(Xpred2, y_downsample):
    
        plt.subplot(231)
        plt.imshow(y_downsample[0].detach().to('cpu').numpy(), origin='lower')
        plt.subplot(232)
        plt.imshow(temp, origin='lower')
        plt.subplot(233)
        plt.imshow(clean, origin='lower')

        plt.subplot(234)
        plt.imshow(y_downsample[0].detach().to('cpu').numpy(), origin='lower')
        plt.subplot(235)
        plt.imshow(temp2, origin='lower')
        plt.subplot(236)
        plt.imshow(clean2, origin='lower')
        plt.show()
        print(loss_fn(Xpred, y_downsample) , loss_fn(Xpred2, y_downsample))

# 保存模型

In [ ]:
torch.save(model.state_dict(), 'model_12epoch.pth')

model = model_unet.UNet(device=device)
model.load_state_dict(torch.load('model_9epoch.pth'))

In [ ]:
try:
    !jupyter nbconvert --to python train_the_model.ipynb
except:
    pass